In [1]:
import numpy
import pickle
import torch
from torch.optim import Adam
import json
import matplotlib as plt
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import random 
import jieba
import Ipynb_importer
import Untitled1
from Untitled1 import load_pretrained_word2vec
#Encoder, Attn, Dynamic_route, Relation_score

from Untitled1 import dict_2_2array, sample_sq, Fewshot


with open('xinwang_train.json') as f:
    load_json = json.load(f)
    data = load_json['rasa_nlu_data']['common_examples']
for i in range(len(data)):
    tep = data[i]['intent']
    data[i]['intent'] = data[i]['text']
    data[i]['text'] = tep
#删除重复项
res_data = []
for item in data:
    if not item in res_data:
        res_data.append(item)

importing Jupyter notebook from Untitled1.ipynb


In [2]:
intentlist = []
for item in res_data:
    if not item['intent'] in intentlist:
        intentlist.append(item['intent'])
textlist = [0 for i in range(len(intentlist))]
for i in range(len(res_data)):
    if res_data[i]['intent'] in intentlist:
        textlist[intentlist.index(res_data[i]['intent'])] += 1 
        #每一个intent下面有多少个同义句

In [3]:
del_data = []
for item in res_data:
    if (textlist[intentlist.index(item['intent'])] >4):
        del_data.append(item)
    else:
        textlist[intentlist.index(item['intent'])]  = 0
print(len(del_data))
data = del_data
#把同义句低于5个的intent包含部分删除，intent包含的是所有的intent集合，text包含的是intent集合对应项的同义句的个数，

8176


In [4]:
import pickle
intent2id = {}
for item in data:
    if not item['intent'] in intent2id:
        intent2id[item['intent']] = len(intent2id)
print(len(intent2id))
with open("intent2id",'wb') as f:
        pickle.dump(intent2id,f) 
        
word2id={}
word2id['PAD'] = 0
word2id['UNK'] = 1
for item in data:
    seg_text = jieba.cut(item['text'],cut_all=False)
    seg_text = (list(seg_text))   
    for w in seg_text:
        if w not in word2id:
            word2id[w] = len(word2id)        
    item['text']=seg_text#使用结巴分词已经划分好
with open("word2id",'wb') as f:
        pickle.dump(word2id,f) 

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


355


Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home1/fangting/anaconda3/envs/ft/lib/python3.6/site-packages/jieba/__init__.py", line 152, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpeyr2txox' -> '/tmp/jieba.cache'
Loading model cost 0.807 seconds.
Prefix dict has been built succesfully.


In [5]:
data1 = data
for item in data1:
    for w in item['text']:
        if w not in word2id:
            w = 'UNK'
for i in range(2):
    print(data1[i])

{'intent': '可以咨询吗？', 'text': ['人工', '服务']}
{'intent': '可以咨询吗？', 'text': ['人工', '客服']}


In [6]:
#转化成数字label
with open('intent2id','rb') as f:
    intent2id = pickle.load(f)
with open('word2id','rb') as f:
    word2id = pickle.load(f)   
for item in data1:
    #print(intent2id[item['intent']])
    item['intent'] = intent2id[item['intent']]
    item['text'] = [word2id[w] for w in item['text']]

In [7]:
#检查text的长度分布,截断长度
text_len = [0 for i in range(100)]
for item in data1:
    if(len(item['text']))<100:
        text_len[len(item['text'])]+=1
#print(text_len)
max_len = 20
for item in data1:
    length = len(item['text'])
    if length<20:
        item['text'].extend([0] * (max_len - length))
    else:
        item['text'] = item['text'][:20]

In [8]:
random.shuffle(data1)
train_data = data1[:7300]
test_data = data1[7300:]
#可用的有len_data，便于lstm进行切分。data1。word2id()，intent2id

In [9]:
#做一个intent的同义句集合(intent_same记录同义句数量)，集合i位置代表intent_i的同义句有多少个，。筛选intent同义句大于15个的data条数，
intent_same = [0 for i in range(355)]
for item in train_data:
    intent_same[item['intent']] += 1
#print(intent_same)
#train_data1筛选出intent同义句>15
train_data1 = []
for item in train_data:
    if intent_same[item['intent']]>15:
        train_data1.append(item)


In [10]:
#构建最终数据
intent2text = {}
intent_texts_len = {}
for item in (train_data1):
    i = item['intent']
    j = item['text']
    if str(i) in intent2text:
        intent2text[str(i)].append(j)
        intent_texts_len[str(i)].append(len(j))
    else:
        intent2text[str(i)] = [j]
        intent_texts_len[str(i)] = [len(j)]


In [11]:
#在每个episode，挑选5个class，每个class中选5个作为example，10个作为query，
#可用的有train_data1,intent2text(dict，intent作为key（char），text是一个二维数组),intent_texts_len
#print(intent_texts_len)


In [16]:
eposide_iter = 50000
way = 5
shot = 5
query = 10
dynamic_iter = 3
k = 100
d_a = 64
emb = 300
hid=128
lr =  0.001
PATH = './model/'
#print(train_classes)

In [17]:
with open('emb_weights','rb') as f:
    emb_weights = pickle.load(f)
with open('word2id','rb') as f:
    word2id = pickle.load(f)  #2889个词
#for i in range(eposide_iter):
from Untitled1 import sample_sq, dict_2_2array
support_idx, query_idx = {},{}
#support_idx, query_idx = sample_sq(intent2text,intent_texts_len,way,shot,query)

In [ ]:
from Untitled1 import dict_2_2array, sample_sq, Fewshot
from torch.optim import Adam
model = Fewshot(num_words=len(word2id),embed_size=emb,hidden_size=hid,emb_weight =emb_weights,
                   way=way,shot=shot,query=query,dynamic_iter=dynamic_iter,k=k,d_a=d_a)
model.cuda()
optim = Adam(lr=lr, params=filter(lambda x: x.requires_grad, model.parameters()),weight_decay=5e-4)

for i in range(eposide_iter): 
    if (i+1)%300==0 and i<=2000:
        lr = lr*0.7
        optim = Adam(lr=lr, params=filter(lambda x: x.requires_grad, model.parameters()),weight_decay=5e-4)
    optim.zero_grad()
    support_idx, query_idx = {},{}
    support_ways_text,support_ways_text_lens = [],[]
    query_ways_text,query_ways_text_lens = [],[]
        
    support_idx, query_idx = sample_sq(intent2text,intent_texts_len,way,shot,query)  
    support_ways_text,support_ways_text_lens = dict_2_2array(support_idx)
    query_ways_text,query_ways_text_lens = dict_2_2array(query_idx)
    support_ways_text = support_ways_text.cuda()
    support_ways_text_lens = support_ways_text_lens#.cuda()
    query_ways_text = query_ways_text.cuda()
    query_ways_text_lens = query_ways_text_lens#.cuda()
    
    loss = model(support_ways_text.t(),support_ways_text_lens,query_ways_text.t(),query_ways_text_lens)
    
    if i%500==0:
        print(loss)
    if (i%100==0):
        print(i,'------------------------')
    loss.backward()
    optim.step()
    if i%50==0:
        torch.save(model.state_dict(), PATH+str(i)+'.pt')
    

tensor(62.8413, device='cuda:0', grad_fn=<AddBackward0>)
0 ------------------------
100 ------------------------
200 ------------------------
300 ------------------------
400 ------------------------
tensor(49.9999, device='cuda:0', grad_fn=<AddBackward0>)
500 ------------------------
600 ------------------------
700 ------------------------
800 ------------------------
900 ------------------------
tensor(39.4358, device='cuda:0', grad_fn=<AddBackward0>)
1000 ------------------------
1100 ------------------------
1200 ------------------------
1300 ------------------------
1400 ------------------------
tensor(37.5445, device='cuda:0', grad_fn=<AddBackward0>)
1500 ------------------------
1600 ------------------------
1700 ------------------------
1800 ------------------------
1900 ------------------------
tensor(26.7017, device='cuda:0', grad_fn=<AddBackward0>)
2000 ------------------------
2100 ------------------------
2200 ------------------------
2300 ------------------------
2400 --

In [15]:

    
    '''
    output,hidden,_ = encoder(support_ways_text.t(),support_ways_text_lens)
    #output是(max_length, batch_size, hidden_size)（先前向后反向）
    #hidden是(n_layers x num_directions, batch_size, hidden_size)
    
    att = Attn(hidden_size, d_a)
    weighted_rep = att(output,output_batch)
    C_K = weighted_rep.view(way,shot,hidden_size*2)
    Classes_vectors = dynamic_route(C_K)
    
    output,hidden,_ = encoder(query_ways_text.t(),query_ways_text_lens)

    query_vectors = att(output,output.size(1))
    ex = Classes_vectors
    ex1 = (ex.repeat(1,50,1))
    k = 100
    print(query_vectors.size(),ex1.size())

    #query_batch = query_vectors.size(0)
    #relation_score = Relation_score(100,query_batch,query_vectors.size(1),query_vectors.size(1))
    loss = torch.tensor(0.0)
    for i in range(way):
        groud_truth = [0.0 for i in range(way*query)]
        groud_truth[query*i:query*(i+1)] = [1.0 for i in range(query)]
        groud_truth = torch.tensor(groud_truth)
        score_i = relation_score(query_vectors,ex1[i]).squeeze()#type,tensor
        loss += ((score_i - groud_truth)**2).sum()
    print(loss)

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-15-5938b30bc76d>, line 28)